# Hello World!

Here's an example notebook with some documentation on how to access CMIP data.

In [1]:
%matplotlib inline

import xarray as xr
import intake

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 

In [2]:
print('hello world!')

hello world!


## Demonstrate how to use `intake-esm`
[Intake-esm](https://intake-esm.readthedocs.io) is a data cataloging utility that facilitates access to CMIP data. It's pretty awesome.

An `intake-esm` collection object establishes a link to a database that contains file locations and associated metadata (i.e. which experiement, model, etc. thet come from). 

### Opening a collection
First step is to open a collection by pointing to the collection definition file, which is a JSON file that conforms to the [ESM Collection Specification](https://github.com/NCAR/esm-collection-spec). 

The collection JSON files are stored locally in this repository for purposes of reproducibility---and because Cheyenne compute nodes don't have Internet access. 

The primary source for these files is the [intake-esm-datastore](https://github.com/NCAR/intake-esm-datastore) repository. Any changes made to these files should be pulled from that repo. For instance, the Pangeo cloud collection is available [here](https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json).

In [3]:
if util.is_ncar_host():
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
col

glade-cmip6-ESM Collection with 687919 entries:
	> 12 activity_id(s)

	> 24 institution_id(s)

	> 47 source_id(s)

	> 66 experiment_id(s)

	> 162 member_id(s)

	> 35 table_id(s)

	> 1027 variable_id(s)

	> 12 grid_label(s)

	> 59 dcpp_init_year(s)

	> 246 version(s)

	> 6667 time_range(s)

	> 687919 path(s)

`intake-esm` is build on top of [pandas](https://pandas.pydata.org/pandas-docs/stable). It is possible to view the `pandas.DataFrame` as follows.

In [4]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,day,pr,gn,NaN,v20190702,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,hfls,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,prsn,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,va,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,tas,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


It is possible to interact with the `DataFrame`; for instance, we can see what the "attributes" of the datasets are by printing the columns.

In [5]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'dcpp_init_year',
       'version', 'time_range', 'path'],
      dtype='object')

### Search and discovery

#### Finding unique entries
Let's query the data to see what models ("source_id"), experiments ("experiment_id") and temporal frequencies ("table_id") are available.

In [6]:
import pprint 
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 66,
                   'values': ['ssp370', 'histSST-piNTCF', 'histSST',
                              'histSST-1950HC', 'hist-1950HC', 'hist-piNTCF',
                              'piClim-NTCF', 'ssp370SST-lowNTCF',
                              'ssp370-lowNTCF', 'ssp370SST', 'hist-bgc',
                              'esm-ssp585', 'amip-future4K', 'amip-m4K',
                              'a4SST', 'aqua-p4K', 'piSST', 'amip-4xCO2',
                              'a4SSTice', 'amip-p4K', 'aqua-control',
                              'aqua-4xCO2', 'abrupt-4xCO2', 'historical',
                              'piControl', 'amip', '1pctCO2', 'esm-piControl',
                              'esm-hist', 'ssp245', 'ssp585', 'ssp126',
                              'hist-GHG', 'hist-aer', 'dcppA-hindcast',
                              'dcppC-hindcast-noPinatubo',
                              'dcppC-hindcast-noElChichon', 'dcppA-assim',
                              'dcp

#### Searching for specific datasets

Let's find all the dissolved oxygen data at annual frequency from the ocean for the `historical` and `ssp585` experiments.

In [7]:
cat = col.search(experiment_id=['piControl', 'historical', 'ssp585'], table_id='Amon', variable_id='ts', grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
33100,CMIP,BCC,BCC-ESM1,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
33458,CMIP,BCC,BCC-ESM1,historical,r1i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
33817,CMIP,BCC,BCC-ESM1,historical,r3i1p1f1,Amon,ts,gn,NaN,v20181214,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
34296,CMIP,BCC,BCC-ESM1,piControl,r1i1p1f1,Amon,ts,gn,NaN,v20181214,185001-230012,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-ESM...
35762,CMIP,BCC,BCC-CSM2-MR,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181115,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
...,...,...,...,...,...,...,...,...,...,...,...,...
686650,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190222,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MRI/...
686809,ScenarioMIP,NUIST,NESM3,ssp585,r2i1p1f1,Amon,ts,gn,NaN,v20190731,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
686832,ScenarioMIP,NUIST,NESM3,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190728,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
686933,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r1i1p1f2,Amon,ts,gn,NaN,v20190823,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...


It might be desirable to get more specific. For instance, we may want to select only the models that have *both* `historical` and `ssp585` data. We coud do this as follows.

In [8]:
models = set(uni_dict['source_id']['values']) # all the models

for experiment_id in ['historical', 'ssp585']:
    query = dict(experiment_id=experiment_id, table_id='Amon', 
                 variable_id='ts', grid_label='gn')  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})

## ensure the CESM2 models are not included (oxygen was erroneously submitted to the archive)
#models = models - {'CESM2-WACCM', 'CESM2'}

models = list(models)
models

['BCC-CSM2-MR',
 'CanESM5',
 'FGOALS-g3',
 'UKESM1-0-LL',
 'MIROC-ES2L',
 'CAMS-CSM1-0',
 'NESM3',
 'MRI-ESM2-0',
 'MIROC6']

In [9]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Amon', 
                 variable_id='ts', grid_label='gn', source_id=models)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
35762,CMIP,BCC,BCC-CSM2-MR,historical,r2i1p1f1,Amon,ts,gn,NaN,v20181115,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
36203,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,ts,gn,NaN,v20181126,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
36640,CMIP,BCC,BCC-CSM2-MR,historical,r3i1p1f1,Amon,ts,gn,NaN,v20181119,185001-201412,/glade/collections/cmip/CMIP6/CMIP/BCC/BCC-CSM...
175414,CMIP,CAS,FGOALS-g3,historical,r2i1p1f1,Amon,ts,gn,NaN,v20190828,195001-195912,/glade/collections/cmip/CMIP6/CMIP/CAS/FGOALS-...
175415,CMIP,CAS,FGOALS-g3,historical,r2i1p1f1,Amon,ts,gn,NaN,v20190828,196001-196912,/glade/collections/cmip/CMIP6/CMIP/CAS/FGOALS-...
...,...,...,...,...,...,...,...,...,...,...,...,...
686650,ScenarioMIP,MRI,MRI-ESM2-0,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190222,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MRI/...
686809,ScenarioMIP,NUIST,NESM3,ssp585,r2i1p1f1,Amon,ts,gn,NaN,v20190731,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
686832,ScenarioMIP,NUIST,NESM3,ssp585,r1i1p1f1,Amon,ts,gn,NaN,v20190728,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/NUIS...
686933,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r1i1p1f2,Amon,ts,gn,NaN,v20190823,201501-210012,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...


### Loading data

`intake-esm` enables loading data directly into an [xarray.Dataset](http://xarray.pydata.org/en/stable/api.html#dataset).

Note that data on the cloud are in 
[zarr](https://zarr.readthedocs.io/en/stable/) format and data on 
[glade](https://www2.cisl.ucar.edu/resources/storage-and-file-systems/glade-file-spaces) are stored as 
[netCDF](https://www.unidata.ucar.edu/software/netcdf/) files. This is opaque to the user.

`intake-esm` has rules for aggegating datasets; these rules are defined in the collection-specification file.

In [10]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})


xarray will load netCDF datasets with dask using a single chunk for all arrays.
For effective chunking, please provide chunks in cdf_kwargs.
For example: cdf_kwargs={'chunks': {'time': 36}}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 18 group(s)


`dset_dict` is a dictionary of `xarray.Dataset`'s; its keys are constructed to refer to compatible groups.

In [11]:
dset_dict.keys()

dict_keys(['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn', 'CMIP.CAMS.CAMS-CSM1-0.historical.Amon.gn', 'CMIP.CAS.FGOALS-g3.historical.Amon.gn', 'CMIP.CAS.FGOALS-g3.ssp585.Amon.gn', 'CMIP.CCCma.CanESM5.historical.Amon.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Amon.gn', 'CMIP.MIROC.MIROC6.historical.Amon.gn', 'CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn', 'CMIP.MRI.MRI-ESM2-0.historical.Amon.gn', 'CMIP.NUIST.NESM3.historical.Amon.gn', 'ScenarioMIP.BCC.BCC-CSM2-MR.ssp585.Amon.gn', 'ScenarioMIP.CAMS.CAMS-CSM1-0.ssp585.Amon.gn', 'ScenarioMIP.CCCma.CanESM5.ssp585.Amon.gn', 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Amon.gn', 'ScenarioMIP.MIROC.MIROC6.ssp585.Amon.gn', 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn', 'ScenarioMIP.MRI.MRI-ESM2-0.ssp585.Amon.gn', 'ScenarioMIP.NUIST.NESM3.ssp585.Amon.gn'])

We can access a particular dataset as follows.

In [12]:
dset_dict['CMIP.CCCma.CanESM5.historical.Amon.gn']

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 64, lon: 128, member_id: 44, time: 1980)
Coordinates:
  * time       (time) float64 15.5 45.0 74.5 ... 6.015e+04 6.018e+04 6.021e+04
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * member_id  (member_id) <U9 'r10i1p1f1' 'r10i1p2f1' ... 'r9i1p1f1' 'r9i1p2f1'
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) float64 dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 -1.406 1.406 1.406 ... 355.8 355.8 358.6
    lat_bnds   (lat, bnds) float64 -90.0 -86.58 -86.58 ... 86.58 86.58 90.0
    ts         (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 1980, 64, 128), meta=np.ndarray>
Attributes:
    realization_index:          9
    external_variables:         areacella
    branch_time_in_child:       0.0
    table_info:                 Creation Date:(20 February 2

### More advanced queries

As motivation for diving into more advanced manipulations with `intake-esm`, let's consider the task of getting access to grid information in the `Ofx` table_id.

In [13]:
cat_fx = col.search(experiment_id=['historical', 'ssp585'], source_id=models, table_id='Ofx', grid_label='gn')
cat_fx.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
294145,CMIP,CCCma,CanESM5,historical,r2i1p1f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
294146,CMIP,CCCma,CanESM5,historical,r2i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
294147,CMIP,CCCma,CanESM5,historical,r2i1p1f1,Ofx,deptho,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
294684,CMIP,CCCma,CanESM5,historical,r5i1p1f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
294685,CMIP,CCCma,CanESM5,historical,r5i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
...,...,...,...,...,...,...,...,...,...,...,...,...
687179,ScenarioMIP,MIROC,MIROC6,ssp585,r1i1p1f1,Ofx,sftof,gn,NaN,v20190627,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...
687180,ScenarioMIP,MIROC,MIROC6,ssp585,r1i1p1f1,Ofx,areacello,gn,NaN,v20190627,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...
687240,ScenarioMIP,MIROC,MIROC6,ssp585,r3i1p1f1,Ofx,sftof,gn,NaN,v20190627,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...
687241,ScenarioMIP,MIROC,MIROC6,ssp585,r3i1p1f1,Ofx,areacello,gn,NaN,v20190627,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/MIRO...


This, however, comes with lots of redundant information.

Additionally, it may be necessary to do more targeted manipulations of the search. For instance, we've found a handful of corrupted files on `glade` and might need to work around loading these. 

As an illustration of this, in the code below, we specify a list of to queries (in this case one) to eliminate.

In [14]:
import numpy as np

# specify a list of queries to eliminate
corrupt_data = [dict(variable_id='areacello', source_id='IPSL-CM6A-LR',
                     experiment_id='historical', member_id='r2i1p1f1')
               ]


# copy the dataframe 
df = cat_fx.df.copy()

# eliminate data
for elim in corrupt_data:
    condition = np.ones(len(df), dtype=bool)
    for key, val in elim.items():
        condition = condition & (df[key] == val)
    df = df.loc[~condition]
df    

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
262740,CMIP,CCCma,CanESM5,historical,r2i1p1f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
262741,CMIP,CCCma,CanESM5,historical,r2i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
263236,CMIP,CCCma,CanESM5,historical,r5i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
263718,CMIP,CCCma,CanESM5,historical,r12i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
264210,CMIP,CCCma,CanESM5,historical,r1i1p2f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/CMIP/CCCma/CanES...
...,...,...,...,...,...,...,...,...,...,...,...,...
557356,ScenarioMIP,CCCma,CanESM5,ssp585,r18i1p1f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/CCCm...
557374,ScenarioMIP,CCCma,CanESM5,ssp585,r11i1p1f1,Ofx,sftof,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/CCCm...
557384,ScenarioMIP,CCCma,CanESM5,ssp585,r6i1p1f1,Ofx,areacello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/CCCm...
557385,ScenarioMIP,CCCma,CanESM5,ssp585,r6i1p1f1,Ofx,thkcello,gn,NaN,v20190429,NaN,/glade/collections/cmip/CMIP6/ScenarioMIP/CCCm...


We then drop duplicates.

In [15]:
df.drop_duplicates(subset=['source_id', 'variable_id'], inplace=True)

Now, since we've only retained one ensemble member, we need to eliminate that column. If we omit this step, `intake-esm` will throw an error, complaining that different variables are present for each ensemble member. Setting the `member_id` column to NaN precludes attempts to join along the ensemble dimension.

After this final manipulation, we copy the `DataFrame` back to the collection object and proceed with loading the data.

In [16]:
df['member_id'] = np.nan
cat_fx.df = df

In [17]:
fx_dsets = cat_fx.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})


xarray will load netCDF datasets with dask using a single chunk for all arrays.
                     For effective chunking, please provide chunks in cdf_kwargs.
                     For example: cdf_kwargs={'chunks': {'time': 36}}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 3 group(s)


In [18]:
fx_dsets.keys()

dict_keys(['CMIP.CCCma.CanESM5.historical.Ofx.gn', 'CMIP.IPSL.IPSL-CM6A-LR.historical.Ofx.gn', 'CMIP.MIROC.MIROC-ES2L.historical.Ofx.gn'])

In [19]:
for key, ds in fx_dsets.items():
    print(ds.data_vars)

Data variables:
    latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    areacello           (j, i) float32 dask.array<chunksize=(291, 360), meta=np.ndarray>
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(45, 2), meta=np.ndarray>
    thkcello            (lev, j, i) float32 dask.array<chunksize=(45, 291, 360), meta=np.ndarray>
    type                |S3 ...
    sftof               (j, i) float32 dask.array<chunksize=(291, 360), meta=np.ndarray>
Data variables:
    nav_lat         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    nav_lon         (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    bounds_nav_lo

## Demonstrate how spin-up a dask cluster

If you expect to require Big Data capabilities, here's how you spin up a [dask](https://dask.org) cluster using [dask-jobqueue](https://dask-jobqueue.readthedocs.io/en/latest/).

The syntax is different if on an NCAR machine versus the cloud.

In [22]:
if util.is_ncar_host():
    from ncar_jobqueue import NCARCluster
    cluster = NCARCluster(project='UCGD0006')
    cluster.adapt(minimum_jobs=1, maximum_jobs=10)
else:
    from dask_kubernetes import KubeCluster
    cluster = KubeCluster()
    cluster.adapt(minimum=1, maximum=10)
cluster

/glade/work/mclong/miniconda3/envs/CMIP6-201910/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [23]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://128.117.181.208:32844 Dashboard: https://jupyterhub.ucar.edu/dav/user/mclong/proxy/34037/status,Cluster Workers: 0 Cores: 0 Memory: 0 B
